In [64]:
! pip install langgraph langsmith langchain langchain_community

In [65]:
from typing import Annotated
from typing_extensions import TypedDict

In [66]:
! pip install arxiv wikipedia

In [67]:
## Working with external toolls:
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun

##Arxiv and wilkipedia tools:

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=300)
arxiv_tool = ArxivQueryRun(api_wrapper = arxiv_wrapper)

api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)
wiki_tool = WikipediaQueryRun(api_wrapper = api_wrapper)

In [68]:
wiki_tool.invoke('who is rajnikanth')

'Page: Rajinikanth\nSummary: Shivaji Rao Gaikwad (born 12 December 1950), known professionally as Rajinikanth, is an Indian actor who predominantly works in Tamil cinema. In a career spanning over five decades, he has done 170 films that includes films in Tamil, Hindi, Telugu, Kannada, Bengali, and Ma'

In [69]:
arxiv_tool.invoke('Attention is all you need')

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with "

In [70]:
tools = [wiki_tool]



In [71]:
## langgraph Application:
from langgraph.graph.message import add_messages
class State(TypedDict):

  messages:Annotated[list,add_messages]

In [72]:
from langgraph.graph import StateGraph,START,END

In [73]:
graph_builder = StateGraph(State)



In [74]:
! pip install langchain_groq

In [75]:
from langchain_groq import ChatGroq


In [76]:
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')

In [77]:
llm = ChatGroq(groq_api_key=groq_api_key,model_name = 'gemma2-9b-It')

In [78]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7b40a0d43790>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7b40a0bc4a10>, model_name='gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [79]:
llm_with_tools = llm.bind_tools(tools=tools)

In [80]:
def chatbot(state:State):

  return {'messages':[llm_with_tools.invoke(state['messages'])]}

In [81]:
from langgraph.prebuilt import ToolNode,tools_condition

In [82]:
# ✅ Register the chatbot function as a node first
graph_builder.add_node('chatbot', chatbot)

# ✅ Register the tool node
tool_node = ToolNode(tools=tools)
graph_builder.add_node('tools', tool_node)

# ✅ Add entry and connections
graph_builder.add_edge(START, 'chatbot')  # Entry point
graph_builder.add_conditional_edges('chatbot', tools_condition)  # Decision logic
graph_builder.add_edge('tools', 'chatbot')  # Loop back
graph_builder.add_edge('chatbot', END)  # End flow

In [83]:
graph = graph_builder.compile()

In [85]:
! pip install Ipython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.7 MB/s eta 0:00:00


In [90]:
pip install langgraph graphviz

In [91]:
from IPython.display import display,Markdown  ,Image
try:
  display(Image(graph.get_graph().draw_mermaid_png()))
except:
  pass

In [93]:
user_input = 'Hi there, My name is subhash'

# stream the graph
events = graph.stream(
    {'messages': [('user', user_input)]},
    stream_mode='values'
)

# iterate over the events and pretty print the last message
for event in events:
    event['messages'][-1].pretty_print()

================================ Human Message =================================

Hi there, My name is subhash
================================== Ai Message ==================================
Tool Calls:
  wikipedia (call_fng9)
 Call ID: call_fng9
  Args:
    query: subhash


/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


================================= Tool Message =================================
Name: wikipedia

No good Wikipedia Search Result was found
================================== Ai Message ==================================

It seems like there isn't a lot of publicly available information about someone named Subhash. Could you tell me more about who you're looking for? Do you know anything else about them, like their profession, location, or any other identifying details?
